In [12]:
import pandas as pd
import numpy as np

data = pd.read_csv("HW2data.csv")
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


HW2 - Cross Validation
本次作業是k-fold cross-validation (繳交期限：10/29 23:55)

1. 資料集如附檔，label是income (<=50K或>50K兩種)
2. 作業要求：
        a.資料前處理 (可參考HW1)
        b.請使用Random Forest進行分類
        c.請寫自行撰寫function進行k-fold cross-validation(不可使用套件)並計算Accuracy
            c-1. input(k, data)，將data切成k份，其中1份當測試集，剩餘k-1份當訓練集建立模型
            c-2. 輪流將k份的每份資料都當 測試集，其餘當訓練集建立模型，因此會進行k次，k次都計算出Accuracy
            c-3. 將k次的Accuracy平均即為output
        參考範例
            def K_fold_CV(k, data):
                 #設定subset size 即data長度/k
                 #設定Accuracy初始值
                 for i in range(k):
                    #設定testing set與training set的資料起始點與結束點
                    #例如資料有100筆，testing set在本次iteration取第1到10筆，則training set為第11到100筆；下次testing set為11~20，training set為21~100 & 1~10
                    #利用training set建立模型，testing set計算出Accuracy累加
                  return Accuracy/k
3. 請計算k=10的Accuracy，並上傳程式碼與報告

In [3]:
data.info()
# 真的沒有空值嗎？
# 不是，通過欄位值統計發現：其實有問號值"?" 在"workclass", "occupation", "native_country"欄位

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [10]:
# 統計各特徵內的 Unique值個數
for i in data.columns:
    print(data[i].value_counts().sort_index(ascending=False))
    print("-"*40)

90     43
88      3
87      1
86      1
85      3
     ... 
21    720
20    753
19    712
18    550
17    395
Name: age, Length: 73, dtype: int64
----------------------------------------
 Without-pay            14
 State-gov            1298
 Self-emp-not-inc     2541
 Self-emp-inc         1116
 Private             22696
 Never-worked            7
 Local-gov            2093
 Federal-gov           960
 ?                    1836
Name: workclass, dtype: int64
----------------------------------------
1484705    1
1455435    1
1366120    1
1268339    1
1226583    1
          ..
19214      1
18827      1
14878      1
13769      1
12285      1
Name: fnlwgt, Length: 21648, dtype: int64
----------------------------------------
 Some-college     7291
 Prof-school       576
 Preschool          51
 Masters          1723
 HS-grad         10501
 Doctorate         413
 Bachelors        5355
 Assoc-voc        1382
 Assoc-acdm       1067
 9th               514
 7th-8th           646
 5th-6th           3

In [13]:
"""
一.空值處理
特徵workclass中有1836筆"?"
occupation中有1843筆"?"
且兩者問號完全是重疊的，很可能是無法在原始資料中呈現的類別，故，先將?也視作一種種類，不處理空值
native_country中有583筆"?"，用眾數美國補
"""
#用眾數補 native_country 中的問號
def clean_col(col):
    col = col.strip()
    col = col.replace("?","United-States")
    return col

#Used a loop to apply the function to each item in the index object and assign it back to 
#the DataFrame.columns attribute.
new_value=[]
for c in data["native_country"]:
    clean_c = clean_col(c)
    new_value.append(clean_c)

data["native_country"]=new_value
data["native_country"].value_counts().sort_index(ascending=False)
# United-States29170 + ?583 = 29753 正確

Yugoslavia                       16
Vietnam                          67
United-States                 29753
Trinadad&Tobago                  19
Thailand                         18
Taiwan                           51
South                            80
Scotland                         12
Puerto-Rico                     114
Portugal                         37
Poland                           60
Philippines                     198
Peru                             31
Outlying-US(Guam-USVI-etc)       14
Nicaragua                        34
Mexico                          643
Laos                             18
Japan                            62
Jamaica                          81
Italy                            73
Ireland                          24
Iran                             43
India                           100
Hungary                          13
Hong                             20
Honduras                         13
Holand-Netherlands                1
Haiti                       

In [14]:
"""
二.income
>50K→1 
<=50K→0
"""
#將 income中，>50K改寫為1、<=50K改寫為0
def clean_col(col):
    col = col.replace(">50K","1")
    col = col.replace("<=50K","0")
    return col

#Used a loop to apply the function to each item in the index object and assign it back to 
#the DataFrame.columns attribute.
new_income = []
for c in data["income"]:
    clean_c = clean_col(c)
    new_income.append(int(clean_c))

data["income"] = new_income
data["income"].value_counts().sort_index(ascending=False)

1     7841
0    24720
Name: income, dtype: int64

In [15]:
# 觀察資料型態、關聯性、等各式統計資料 in "your_report.html"
from pandas_profiling import ProfileReport

profile = ProfileReport(data, title='Pandas Profiling Report')
profile.to_file("your_report.html")

In [17]:
data.corr()


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income
age,1.000000,-0.076646,0.036527,0.077674,0.057775,0.068756,0.234037
fnlwgt,-0.076646,1.000000,-0.043195,0.000432,-0.010252,-0.018768,-0.009463
education_num,0.036527,-0.043195,1.000000,0.122630,0.079923,0.148123,0.335154
capital_gain,0.077674,0.000432,0.122630,1.000000,-0.031615,0.078409,0.223329
capital_loss,0.057775,-0.010252,0.079923,-0.031615,1.000000,0.054256,0.150526
hours_per_week,0.068756,-0.018768,0.148123,0.078409,0.054256,1.000000,0.229689
income,0.234037,-0.009463,0.335154,0.223329,0.150526,0.229689,1.000000


- education: 是education_num欄的文字版，刪去
- fnlwgt: 相關系數太低，且疑似是計數欄，刪去

- capital_gain: 
- capital_loss: 這兩項刪去反而使Accuracy、Recall、Precision三個都變差，不刪

- 以下兩個先不處理，可能各項特徵與y值的相關性都太低，刪去任何一個都會讓精度下降
- hours_per_week
- age: 17~22,23~30,31~40,.....

In [18]:
#刪去 ["education"]、["fnlwgt"]
df=data  #要進行不可逆的資料破壞→用複製品
df = df.drop(['education', "fnlwgt"], axis=1)


In [19]:
# 把類別型資料轉成數字
from sklearn.preprocessing import LabelEncoder
LE= LabelEncoder()
for i in df.columns:
    if df[i].dtype =='O':
        df[i] = LE.fit_transform(df[i])
df

,age,workclass,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,7,13,4,1,1,4,1,2174,0,40,38,0
1,50,6,13,2,4,0,4,1,0,0,13,38,0
2,38,4,9,0,6,1,4,1,0,0,40,38,0
3,53,4,7,2,6,0,2,1,0,0,40,38,0
4,28,4,13,2,10,5,2,0,0,0,40,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,12,2,13,5,4,0,0,0,38,38,0
32557,40,4,9,2,7,0,4,1,0,0,40,38,1
32558,58,4,9,6,1,4,4,0,0,0,40,38,0
32559,22,4,9,4,1,3,4,1,0,0,20,38,0


In [21]:
#手刻 CV + k=10 + RandomForest 模型
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score

def K_fold_CV(k, data):
    validation_size = int(len(data)/k)
    accuracy=[]
    recall=[]
    precision=[]

    for i in range(k):
        validation = df.iloc[i*validation_size:validation_size*(i+1),:]
        train = df.drop([i*validation_size,validation_size*(i+1)])
        validation_X = validation.drop('income', axis=1)
        validation_y = validation['income'] 
        train_X = train.drop('income', axis=1)
        train_y = train['income']
        
        clf = RandomForestClassifier(n_estimators=300, max_depth=4, n_jobs=-1)
        clf.fit(train_X, train_y)
        
        y_pred_rf = clf.predict(validation_X)
        accuracy.append(accuracy_score(validation_y, y_pred_rf))
        Accuracy_mean = np.mean(accuracy)
        
        recall.append(recall_score(validation_y, y_pred_rf))
        Recall_mean = np.mean(recall)
        
        precision.append(precision_score(validation_y, y_pred_rf))
        Precision_mean = np.mean(precision)
                
    return(Accuracy_mean, Recall_mean, Precision_mean)

Accuracy = K_fold_CV(10,data)

#因為前處理做很少，但Accuracy卻很高，把 Recall, Precision叫出來看
print("Accuracy: ",Accuracy[0])
print("Recall: ",Accuracy[1])
print("Precision: ",Accuracy[2])
# 發現 recall很低，可能與income中，<=50K或>50K兩種比例為 1:3.~有關

Accuracy:  0.847911547911548
Recall:  0.4843354910627241
Precision:  0.8065721427501966
